### Kubernetes and TensorFlow Serving for Machine Learning Zoomcamp

This notebook demonstrates a workflow for:

1. Building a Docker image of a model.
2. Testing the model locally within a container.
3. Creating and managing a local Kubernetes cluster using Kind.
4. Deploying the model to the Kubernetes cluster.
5. Exposing the model via a Kubernetes Service.
6. (Optional) Setting up and observing a Horizontal Pod Autoscaler (HPA)

**Prerequisites:**
- Docker installed and running
- Kubernetes CLI (kubectl) installed
- Kind installed
- Python 3.11.5 (adjust as needed)

### Checking versions

!docker --version
!kubectl version --client
!kind --version

### Cloning the Repository

We will clone the repository to access the base code and `q6_test.py`.

!git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
%cd machine-learning-zoomcamp/cohorts/2024/05-deployment/homework

### Building the Docker Image

We define environment variables for the image name and tag.

IMAGE_NAME = "zoomcamp-model"
IMAGE_TAG = "3.11.5-hw10"
FULL_IMAGE_NAME = f"{IMAGE_NAME}:{IMAGE_TAG}"
FULL_IMAGE_NAME

### Building the Docker image locally

!docker build -t {FULL_IMAGE_NAME} .

### Testing the Image Locally

We will run the container and use `q6_test.py` to validate the model’s response.

!docker run -d --name subscription_test -p 9696:9696 {FULL_IMAGE_NAME}

!python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.7567...}

### Stopping and removing the test container

!docker stop subscription_test
!docker rm subscription_test

### Creating and Testing the Kubernetes Cluster with Kind

Now, we will create a local Kubernetes cluster using Kind.

!kind create cluster

!kubectl cluster-info
!kubectl get services

Expected Output: Check the kubernetes service with type ClusterIP.

### Loading the Docker Image into the Kind Cluster

We need to make the locally built image available inside the Kind cluster.

!kind load docker-image {FULL_IMAGE_NAME}

### Creating the Kubernetes Deployment

In [ ]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  replicas: 1
  selector:
    matchLabels:
      app: subscription
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: zoomcamp-model:3.11.5-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"
          limits:
            memory: "128Mi"
            cpu: "200m"
        ports:
        - containerPort: 9696

!kubectl apply -f deployment.yaml
!kubectl get deployments
!kubectl get pods

### Creating the Service

%%writefile service.yaml
apiVersion: v1
kind: Service
metadata:
  name: subscription-service
spec:
  type: LoadBalancer
  selector:
    app: subscription
  ports:
    - port: 80
      targetPort: 9696

!kubectl apply -f service.yaml
!kubectl get services

## Testing the Service via Port-Forwarding

Since `LoadBalancer` with Kind typically won't have an external IP, we'll use port forwarding:

import subprocess, time

port_forward_process = subprocess.Popen(["kubectl", "port-forward", "service/subscription-service", "9696:80"])
time.sleep(5)

!python q6_test.py

In [ ]:
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}

port_forward_process.terminate()

### Optional: Autoscaling with Horizontal Pod Autoscaler (HPA)

!kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

!kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3
!kubectl get hpa

### Generate load and observe scaling

# In another terminal/environment:
# while True:
#     sleep 0.1
#     python q6_test.py

# kubectl get hpa subscription-hpa --watch

### Cleanup

Remove the Kind cluster after testing:

!kind delete cluster